In [34]:
import os
import time
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import padding

# Exercício 1

#### Neste exercício é pedido que se crie uma comunicação privada assíncrona entre um agente Emitter e um agente Receiver que cubra alguns aspetos enunciados a seguir.

Começamos por desenvolver uma função para derivar uma chave, baseada no funcionamento do protocolo DH para geração de chaves privadas e públicas. Para este protocolo, é necessário gerar parâmetros que devem ser os mesmos para os dois agentes e podem ser reutilizados.<br>
Inicialmente, gera-se a chave privada e depois é feita a troca com uma chave pública recebida (que será do outro agente com quem se está a comunicar) que irá gerar uma chave partilhada (a mesma nos dois agentes). Esta chave é ainda passada a uma função de derivação de chaves, para permitir uma mistura de informação que a torna mais segura.<br>
Para a chave poder ser gerada, é preciso saber a chave pública do outro agente, para isso, foi também criada uma função que devolve a chave pública gerada.

In [2]:
parameters = dh.generate_parameters(generator=2, key_size=1024)

def get_public_key(private_key):
        return private_key.public_key()

def derivate_key(private_key,public):
        shared_key = private_key.exchange(public)
        
        derived_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info=b'handshake data',
        ).derive(shared_key)

        return derived_key

Depois da chave gerada, é o momento de enviar a mensagem para o agente Receiver.<br>
Para isso, é primeiro gerada uma assinatura, usando o HMAC que será concatenada com a mensagem cifrada e enviada ao Receiver para que este possa verificar o remetente.

In [3]:
def auth(message,derived_key):
        h = hmac.HMAC(derived_key, hashes.SHA256())
        h.update(message)
        return h.finalize()

De seguida, é usada a função *urandom()* do pacote *os* do *python* para gerar o *nonce* necessário para cifrar a mensagem.<br>
A cifra usada foi a AESGCM. Depois de tudo gerado e da mensagem cifrada, esta é concatenada à assinatura e ao *nonce* e enviada para o Receiver.

In [43]:
def cifraGCM(nonce,message,derived_key):
    aesgcm = AESGCM(derived_key)
    return aesgcm.encrypt(nonce, message, b'some associated data')

def send_message(message,derived_key):
        signature = auth(b'this is a message to check the signature',derived_key)
        message = message.encode('utf-8')
        nonce = os.urandom(16)
        ct = cifraGCM(nonce,message,derived_key)
        
        return signature + nonce + ct

Do lado do Receiver, os dados são recebidos e desempacotados.

In [44]:
def unpack_data(dados):
      signature = dados[0:32]
      nonce = dados[32:32+16]
      ct = dados[32+16:]

      return signature, nonce, ct

Depois de ter os dados desempacotaos e separados, é necessário fazer a verificação da assinatira da mensagem recebida. Novamente, usando o HMAC.

In [45]:
def verify(signature,derived_key):
      h = hmac.HMAC(derived_key, hashes.SHA256())
      h.update(b'this is a message to check the signature')
      return h.verify(signature)

Se a assinatura for verificada, passa-se então à decifragem da mensagem pela cifra AESGCM.

In [46]:
def decifraGCM(nonce, ct, derived_key):
    aesgcm = AESGCM(derived_key)
    return aesgcm.decrypt(nonce, ct, b'some associated data')

def read_message(ct, derived_key):
      signature, nonce, ct = unpack_data(ct)
      try :
          verify(signature, derived_key)
      except:
          raise Exception("Falha na autenticidade da chave") 

      texto_limpo = decifraGCM(nonce, ct, derived_key)
    
      return texto_limpo.decode('utf-8')

Finalmente, enviamos a mensagem do Emitter para o Receiver.

In [47]:
def main():
    rv_private_key = parameters.generate_private_key()
    em_private_key = parameters.generate_private_key()
    em_derived_key = derivate_key(em_private_key,get_public_key(rv_private_key))
    rv_derived_key = derivate_key(rv_private_key,get_public_key(em_private_key))
    
    dados = send_message("Estruturas Criptográficas é uma unidade curricular do perfil de CSI", em_derived_key)
    print('encrypted text:',dados)

    try:
        pt = read_message(dados,rv_derived_key)

        print('decrypted text:', pt)
    except:
        print("Falha na autenticação da chave")  

main()

encrypted text: b"g A\xd8\xde2z\xdf\xa9\xe0\x82%+\xa3Y\x9a \x1c\x91\xcc#\\\xf9ti\x02z\xf5\x9cm\x15\x81\xdb\x94\x81\x14\xd8r'\xeb\xe5\x9f\x07G\xe7{\x1d\xad*\xcb\t\x98X\xb2]cD*Yk#\x8c\xeb\xd2\xf6\xe6\xf6\xda$\xf3Q\xbc\x93\x0f\x8d\x1c}Z\xa7\x1f\xec\x93\x05\x1b\x8c\xe1\xcc<0^\xdf\xa1\xc1K\x014\xeb\x83\xc1\xc5\xf2\x84F\x04\xb3\xc1\xbba8\xa2\x16\x92\xf0\xc9\xa6\xf2\xe3\xf9\x92\xb9\xdd\xe2\xd1\xe3t\xdd\x01\xbej\xf9a\x8f\xd7"
decrypted text: Estruturas Criptográficas é uma unidade curricular do perfil de CSI


# Exercicio 2
## Criar uma cifra com autenticação de meta-dados a partir de um PRG


Como objetivo inicial, esperava-se que o nosso grupo implementasse um gerador pseudo-aleatório do usando uma função XOF("extended output function") - SHAKE256 - de forma a gerar uma sequência de palavras, cada uma com 64 bits.


Para começar, criamos uma função que recebe a password e a partir de uma *KDF* - no nosso caso decidimos escolher a **PBKDF2HMAC**, mas também poderíamos ter usado a *HKD*. Esta decisão foi tida em conta com a leitura da documentação da lib *Crypthography*, a qual inferia que há diferentes tipos de funções de derivação para diferentes tipos de propósitos. Como o nosso objetivo seria criar uma seed para o gerador, escolhemos um dos que era recomendado. 

In [61]:
def derive_key(password):
    salt = os.urandom(16)
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        )

    return kdf.derive(password)

De seguida, e conforme enunciado, era necessário criar o gerador, que tivesse um limite de palavras definido por um valor inicialmente parametrizado como **N**. Com a seed gerada pela *key derivation function* que escolhemos seria possível gerar 2^N palavras de 64 bits cada.
Seguindo o requisitado para este projeto, era necessário usar o **SHAKE256** para definir o gerador - como se sabe o *SHAKE256* é uma *extendable-output function* e define-se o seu uso tendo em conta a necessidade de autenticação do criptograma e geração de palavras usando uma XOF. 


In [62]:
def prg(seed,N):
    digest = hashes.Hash(hashes.SHAKE256(8 * pow(2,N))) # sequencia palavras 64 bits / 8 = 8 bytes
    digest.update(seed)
    msg = digest.finalize()
    return msg

A partir da sequência de bits representativa da lista de palavras, gerada pelo PRG, conseguimos aplicar a cifra **OTP**.
Assumindo a variante construída em 1919, é necessário definir a operação *XOR* em contraste com a sua versão original, criada com adição modular. É então preciso aplicar esta operação ao processo de cifragem e decifragem, sendo que, a partir de duas sequências de bytes é aplicado o XOR a ambas.


In [63]:
BLOCK = 8 # bytes; representativo de cada bloco que define uma palavra (64 bits)

In [64]:
def pad_divide(message):
    x = []
    for i in range (0,len(message), BLOCK):
        next = i+BLOCK
        x.append(message[i:next])
    return x

É importante denotar o mecanismo associado a estes processos de cifragem e decifragem. Inicialmente é feito o *padding*, de forma a que os blocos de mensagens sejam garantidamente múltiplos. Após isto, e para simplicidade prática,é necessário dividir a mensagem em blocos de 8 bytes, cada um representando uma palavra.

In [65]:
def cipher(k,msg):
    ciphertext = b''
    pad = padding.PKCS7(64).padder()
    
    padded = pad.update(msg) + pad.finalize()
    p = pad_divide(padded)

    for x in range (len(p)): # Percorre blocos do texto limpo
        for bloco, byte in enumerate(p[x]): # Percorre bytes do bloco do texto limpo
            ciphertext += bytes([byte ^ k[x:(x+1)*BLOCK][bloco]]) # xor of 2 bit sequences plain text and cipher_key
    return ciphertext


In [66]:

def decipher(k,ciphertext):
    plaintext=b''
    
    p=pad_divide(ciphertext)

    for x in range (len(p)): # Percorre blocos do texto cifrado
        for bloco, byte in enumerate(p[x]): # Percorre bytes do bloco do texto cifrado
            plaintext += bytes([byte ^ k[x:(x+1)*BLOCK][bloco]]) 
    
    # Algoritmo para retirar padding para decifragem
    unpadder = padding.PKCS7(64).unpadder()
    # Retira bytes adicionados 
    unpadded = unpadder.update(plaintext) + unpadder.finalize()
    return unpadded.decode("utf-8")

In [67]:
    
cipher_key = derive_key(b'password')
msg = prg(cipher_key,2)
mensagem = b"Ultra secret message"

ct = cipher(msg,mensagem)
dt = decipher(msg,ct)


print("OG TEXT: ", mensagem)
print("CT:  ", ct)
print("DT:  ", dt)

OG TEXT:  b'Ultra secret message'
CT:   b'\xe0\x1bp\x06\xfb\x0cR\xda\x14v\x11\xee\x0cL\xda>w\x15\xfdI%\xbbIM'
DT:   Ultra secret message


# Exercício 3

Para proceder à comparação das duas cifras, desenvolveu-se um cenário de teste onde é testado o tempo que cada uma das cifras demora a cifrar o mesmo texto.

In [81]:
N = 10
    
def fst_cipher():
    pwd = b"password"
    key = derive_key(pwd)
    nonce = os.urandom(16)
    plaintext = os.urandom(2 ** N)
    ciphertext = cifraGCM(nonce, plaintext, key)
    return ciphertext

def snd_cipher():
    pwd = b"password"
    key = derive_key(pwd)
    plaintext = os.urandom(2 ** N)
    words = prg(key,N)
    ciphertext = cipher(words,plaintext)
    return ciphertext

start = time.time_ns()
fst_text = fst_cipher()
stop = time.time_ns()
print('encrypted text:', fst_text)
print('elapsed time fst:', stop-start, 'ns')

start = time.time_ns()
snd_text = snd_cipher()
stop = time.time_ns()
print('encrypted text:', snd_text)
print('elapsed time snd:', stop-start, 'ns')

encrypted text: b'\x8e\x06\x9e@G\xe5\xd6\x1a\xc0\x08\xce\xf9}3\xbd\x83\x15rN\xedN\xd3D\xa3\x00?\xa4\xc0=\xa7$\xc45\x8c\xa0\x9b\xb7\x9d\xe9[^\xa1\x06,\xe4\x8b\xd1\xbb<!\xd3\x12\xac\xd85\xaa\x85\xc6\x1f\x88c\xd8P,8W\x07\xbbZ&\x89 \xd8?+E\xf3\x84\xe8\x19X\xcb\xca\x8ec]]\x17\x88\xa7#\xee\xd4\xf7>|\xa9\x059\xda1S>\xb18\x87b\xc5B\x02\xc3J`\x95\x1a\xba\xd9,\x9c\xb0\xb6\xd4\xd9?\x7f\xe9\xa2\x84A\x81\x91\xf0\x16\xff\'\xce\xe3\xdd\xff\xf4\xf0X\xf5\x8cOhO!\x96+\xac\x9cFJ*\x03\xfa\xab\xc4P\x80\xfc0\xec\xb3f\x99:A\xdag\xd8\xf20[&\xa8\xe0u.\xaeB9E\xe4q\xf9(\xc8V\xda\xcb\xe6\xc9\xb3\xcd\xf9\xf6\xc5\x9f\xb65\xd7\x7f\xa2q\x8d\x031`\xd0\x10\xa6%\x04\xfa\x03\xd2C\xc8v5\x99\xf9\xa6\x17\x8e7\xa3\x02A\xbb\xfb\xa5\x02H@[x\xf8\x15JR\xef\x1a2+\xc5n\x07`\xfd\xd5\xf4\xa5u[E\xeb\xa7\x84I\xc0\xb4q\xdd\x8f\xeaA\xa5\xea\xb2\x0c}\x95\x97K\xde\xbd\xa2\xad\x1f\xf9\xabZ*\x9e\xcb\x11\x1b\r\xd0r\xc0\x97\xb5\xcee\xe0\x16\x9d\xb2\x9a\x83\x8b\xff\x88\x8f\xedf\xe4S\xea9\xed\xb1\xe4(F\xdf\\Ft\x1c\xfbn>\xfcN\x1e&>\x80n\xf2\xf3T

Para concluir, 